# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv
from dotenv import load_dotenv
import os 

In [2]:
load_dotenv()
token = os.getenv('4sqtoken')

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [3]:

url1 = "https://api.foursquare.com/v3/places/search?ll=41.3977421%2C2.190249315&categories=13032&limit=10"

headers = {"accept": "application/json",
            "Authorization": token }

response1 = requests.get(url1, headers=headers)

print(response1.text)

{"results":[{"fsq_id":"5b8521d489e490002cefef18","categories":[{"id":13035,"name":"Coffee Shop","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}},{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":567,"geocodes":{"main":{"latitude":41.396989,"longitude":2.183409},"roof":{"latitude":41.396989,"longitude":2.183409}},"link":"/v3/places/5b8521d489e490002cefef18","location":{"address":"Ausiàs Marc, 151","admin_region":"Cataluña","country":"ES","formatted_address":"Ausiàs Marc, 151, 08013 Barcelona Catalonia","locality":"Barcelona","postcode":"08013","region":"Catalonia"},"name":"Three Marks Coffee","related_places":{},"timezone":"Europe/Madrid"},{"fsq_id":"556d6217498ef50c5d323048","categories":[{"id":13003,"name":"Bar","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/pub_","suffix":".png"}},{"id":13035,"name":"Coffee Shop","icon":{"pref

In [4]:
jason = response1.json()
results = jason['results']
df = pd.DataFrame(results)
df_name_location = df[['name','location']]
df_name_location



,name,location
0,Three Marks Coffee,"{'address': 'Ausiàs Marc, 151', 'admin_region'..."
1,Nømad Roaster's Home,"{'address': 'Calle de Pujades, 95', 'admin_reg..."
2,Gocce di latte,"{'address': 'Plaza de Palau, 4', 'admin_region..."
3,Nømad Coffee Lab & Shop,"{'address': 'Pasatge Sert, 12', 'admin_region'..."
4,Roast Club,"{'address': 'Carrer de València, 190', 'admin_..."
5,Dalston Coffee,"{'address': 'Calle de Les Ramelleres, 16', 'ad..."
6,SlowMov,"{'address': 'Carrer de Luis Antúnez, 18', 'adm..."
7,Syra Coffee,"{'address': 'Siracusa, 13', 'admin_region': 'C..."
8,Satan's Coffee,"{'address': 'Gran Vía de Les Corts Catalanes, ..."
9,Brew Coffee,"{'address': 'Calle de Roger de Flor, 102', 'ad..."


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [5]:
url2 =  "https://api.foursquare.com/v3/places/search?ll=41.3977421%2C2.190249315&radius=500&categories=13032&limit=5"
response2 = requests.get(url2, headers=headers)
print (response2.text)


{"results":[{"fsq_id":"530c5e6911d2be0b8d22e27a","categories":[{"id":13034,"name":"Café","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cafe_","suffix":".png"}},{"id":13035,"name":"Coffee Shop","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}},{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":135,"geocodes":{"main":{"latitude":41.396876,"longitude":2.191489},"roof":{"latitude":41.396876,"longitude":2.191489}},"link":"/v3/places/530c5e6911d2be0b8d22e27a","location":{"address":"Calle Pamplona, 88","admin_region":"Cataluña","country":"ES","cross_street":"C. de Pallars","formatted_address":"Calle Pamplona, 88 (C. de Pallars), 08018 Barcelona Catalonia","locality":"Barcelona","postcode":"08018","region":"Catalonia"},"name":"Skye coffee co","related_places":{},"timezone":"Europe/Madrid"},{"fsq_id":"555f2357498ee6528074710b","categories":[{"

In [32]:
json_response2 = response2.json()
results2 = json_response2['results']
df2 = pd.DataFrame(results2)
df_name_location2 = df2[['name','geocodes']]
df_name_location2


,name,geocodes
0,Skye coffee co,"{'main': {'latitude': 41.396876, 'longitude': ..."
1,Espai Joliu,"{'main': {'latitude': 41.398735, 'longitude': ..."
2,Nømad Roaster's Home,"{'main': {'latitude': 41.397977, 'longitude': ..."
3,Syra Coffee Poblenou,"{'main': {'latitude': 41.396518, 'longitude': ..."
4,Market Cuina Fresca,"{'main': {'latitude': 41.398365, 'longitude': ..."


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [33]:


def foursquare_places (venue, lat, long):
    url3 = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{long}&limit=10"
    response3 = requests.get(url3, headers=headers)
    json_response3= response3.json()
    results3 = json_response3['results']
    df3 = pd.DataFrame(results3)
    df_name_location3 = df3[['name','geocodes']]


    return df_name_location3
function_response = foursquare_places('coffe',41.3977421,2.190249315)

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [39]:
function_response.to_json(r'../lab-coffee-shops-sergi/data/coffe_shope.json')


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [36]:
import pymongo 


In [40]:
client = pymongo.MongoClient('localhost:27017')
dabatase = client['ironhack']
collection = dabatase.get_collection('ironcoffee')

In [42]:
with open ('../lab-coffee-shops-sergi/data/coffe_shope.json') as f:
    data = json.load(f)
data


{'name': {'0': "Satan's Coffee Corner",
  '1': 'Nømad Coffee Lab & Shop',
  '2': 'Cafés el Magnífico',
  '3': "Satan's Coffee",
  '4': 'Hidden Café Barcelona',
  '5': 'Roast Club',
  '6': 'Espai Joliu',
  '7': 'Federal',
  '8': 'Syra Coffee',
  '9': 'Dalston Coffee'},
 'geocodes': {'0': {'main': {'latitude': 41.38264, 'longitude': 2.175303},
   'roof': {'latitude': 41.38264, 'longitude': 2.175303}},
  '1': {'main': {'latitude': 41.388989, 'longitude': 2.177167},
   'roof': {'latitude': 41.388989, 'longitude': 2.177167}},
  '2': {'main': {'latitude': 41.383541, 'longitude': 2.180957},
   'roof': {'latitude': 41.383541, 'longitude': 2.180957}},
  '3': {'main': {'latitude': 41.393575, 'longitude': 2.174295},
   'roof': {'latitude': 41.393575, 'longitude': 2.174295}},
  '4': {'main': {'latitude': 41.388351, 'longitude': 2.135953},
   'roof': {'latitude': 41.388351, 'longitude': 2.135953}},
  '5': {'main': {'latitude': 41.387825, 'longitude': 2.157221},
   'roof': {'latitude': 41.387825, 'l

In [43]:
dataframeto = pd.DataFrame(data)

In [44]:
dict_ = dataframeto.to_dict(orient='records')

In [45]:
collection.insert_many(dict_)